The N-Queens problem asks you to plave N queens on an NxN chessboard so that no queen attacks another.

This example shows that the min conflicts solver can find solutions to the N-Queens problem faster than the backtracking CSP approach for large N

In [32]:
# this may be needed for remote execution
!pip install --user python-constraint

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [26]:
from constraint import *
import time
# Use plotly for graphs in a notebook
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True) 

In [27]:
class NQ(Problem):

    def __init__(self, size=8, solver=None):
        super(NQ, self).__init__(solver=solver)

        self.addVariables(range(size), range(size))
        for c1 in range(size):
            for c2 in range(c1+1,size):
                self.addConstraint(lambda row1, row2, col1=c1, col2=c2:
                                abs(row1-row2) != abs(col1-col2) and row1 != row2,
                                (c1, c2))


In [69]:
# how many seconds is too long?
too_long = 100
# should we skip backtrack because it will take too long?
skip = False
# list of times for MinCongflictsSolver
mcs_times = []
# problem sizes we want to explore
sizes = list(range(25,501,25))

In [70]:
print('size', 'backtrack', 'minConflicts')

for size in sizes:
    bts = NQ(size, solver=BacktrackingSolver())
    mcs = NQ(size, solver=MinConflictsSolver())
    t0 = time.time()
    # do BacktrackingSolver id we've not given up due to the last one taking too long
    if not skip:
        bts.getSolution()
        t1 = time.time()
        skip = t1 - t0 > too_long
    else:
        t1 = t0
    # now try MinConflictsSolver
    mcs.getSolution()
    t2 = time.time()
    mcs_times.append(t2-t1)
    print('{}\t{:.2f}\t{:.2f}'.format(size, t1-t0, t2-t1))
    
    

size backtrack minConflicts
25	0.02	0.03
50	0.20	0.23
75	1.73	0.40
100	0.29	1.05
125	0.77	1.91
150	365.84	2.74
175	0.00	6.34
200	0.00	9.07
225	0.00	9.91
250	0.00	13.26
275	0.00	18.81
300	0.00	33.40
325	0.00	27.59
350	0.00	29.72
375	0.00	42.73
400	0.00	55.98
425	0.00	67.52
450	0.00	70.81
475	0.00	82.97
500	0.00	115.89


In [71]:
# plot the results for MinConflictsSolver
data = go.Data([go.Scatter(x=sizes, y=mcs_times)])
layout = go.Layout(title='Execution time for N-queens problems of different sizes',
                   xaxis={'title':'problem size'}, 
                   yaxis={'title':'seconds'})
iplot(go.Figure(data=data, layout=layout))